In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00


In [1]:
from transformers import TFAutoModelForSequenceClassification
from transformers import TFRobertaModel
from transformers import AutoTokenizer, RobertaTokenizer

c:\Users\hesse\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_corpus(corpus_file):
    '''
    The read_corpus() function is used to load data (training or testing data)
    from a file into separate variables that store the tokens from the document
    and the associated labels.
    '''
    documents = []
    labels = []

    with open("./data/"+corpus_file, encoding='utf-8') as in_file:
        for line in in_file:
            tokens = line.strip()
            documents.append(" ".join(tokens.split()[:-1]).strip())

            # documents.append(tokens[:-1])
            labels.append(tokens.split()[-1])
    return documents, labels

In [ ]:
!ls
%cd gdrive/MyDrive/LFD_project
!ls

gdrive	sample_data
/content/gdrive/MyDrive/LFD_project
data  lfd_project_pretrained.ipynb


In [3]:
X_train, Y_train = read_corpus("train.tsv")
X_dev, Y_dev = read_corpus("dev.tsv")
X_test, Y_test = read_corpus("test.tsv")


lm = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(lm)
model = TFAutoModelForSequenceClassification.from_pretrained(lm, num_labels=1)
tokens_train = tokenizer(X_train, padding=True, max_length=100,
               truncation=True, return_tensors="np").data
print(tokens_train["input_ids"].shape)
tokens_dev = tokenizer(X_dev, padding=True, max_length=100,
             truncation=True, return_tensors="np").data
tokens_test = tokenizer(X_test, padding=True, max_length=100,
             truncation=True, return_tensors="np").data

c:\Users\hesse\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hesse\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the 

(12240, 100)


In [5]:
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import numpy as np


In [6]:
encoder = LabelBinarizer()
Y_train_bin = encoder.fit_transform(Y_train)
Y_dev_bin = encoder.fit_transform(Y_dev)
Y_test_bin = encoder.fit_transform(Y_test)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

learning_rate = 5e-5

loss_function = BinaryCrossentropy(from_logits=True)

optim = SGD(learning_rate)

model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
model.fit(tokens_train, Y_train_bin, verbose=2, epochs=5, callbacks=[callback], batch_size=16, validation_data=(tokens_dev, Y_dev_bin))

Epoch 1/5


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.metrics import classification_report


Y_pred_logits = model.predict(tokens_test).logits
Y_pred_labels = np.argmax(Y_pred_logits, axis=1)
label_mapping = {0: 'NOT', 1: 'OFF'}
Y_pred_labels = [label_mapping[label] for label in Y_pred_labels]
Y_test_labels = [label_mapping[label] for label in np.argmax(Y_test_bin, axis=1)]
class_report = classification_report(Y_test_labels, Y_pred_labels, target_names=label_mapping.values())
print(class_report)

19/19 [==============================] - 8s 254ms/step


In [ ]:
len(X_train)

4800

In [ ]:
Y_pred = model.predict(tokens_dev)["logits"]

19/19 [==============================] - 7s 209ms/step


In [ ]:
Y_pred = model.predict(tokens_test)["logits"]

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(Y_test, Y_pred, average = "macro")
print(f"Final F1 score: {f1}")